# [모듈 8.1] 엔드포인트 배포 스텝 개발 (SageMaker Model Building Pipeline 배포 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 30분 소요 됩니다.

- 0. 모델 엔드포인트 배포 개요 
- 1. 기본 세이지 메이커 호스트 및 로컬 세션 정의
- 2. 엔드포인트 배포 스텝 개발 및 실행
    - 아래의 3단계를 진행하여 SageMaker Model Building Pipeline 에서 배포 스텝 개발 함. 아래의 (1), (2) 단계는 옵션이지만, 실제 현업 개발시에 필요한 단계이기에 실행을 권장 드립니다.
        - (1) [옵션] **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 엔드포인트를 생성
        - (2) [옵션] 로컬에서 Endpoint 생성 스크립트 실행 (Boto3 API 이용)
        - (3) [필수] SageMaker Model Building Pipeline 에서 엔드포인트 생성을 수행합니다.

---

# 0. 모델 엔드포인트 배포 개요 

---
## SageMaker 호스팅 아키텍쳐
- 일반적인 아키텍쳐의 그림이고, 오토 스케일링이 적용이 되어 있습니다.

![sagemaker-endpoints.png](img/sagemaker-endpoints.png)

## SageMaker 호스팅 3단계 

이 노트북에서는 "SageMaker 호스팅 서비스를 사용하여 모델 배포는 다음과 같은 3단계 프로세스" 중에 두번째와 세번째 단계를 진행 합니다.
- ~(1) SageMaker 에서 모델 생성~
- (2) HTTPS endpoint에 대한 endpoint configuration 만들기
- (3) HTTPS endpoint에 대한 endpoint configuration 만들기

![hosting-architecture.png](img/hosting-architecture.png)


### ~(1) SageMaker 에서 모델 생성 (이 노트북에서 포함하는 내용임)~

### (2) HTTPS endpoint에 대한 endpoint configuration 만들기
- 용어 정의: 프로덕션 변형(Product Varaiant)
    - 호스팅할 모델과 해당 모델을 호스팅하기 위해 선택한 리소스를 식별하는 객체 임. 
        - 정의 --> [ProductionVariant](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/APIReference/API_ProductionVariant.html)
    
    
- 프로덕션 변형(Product Varaiant) 에서 하나 이상의 모델 이름과 SageMaker 에서 각 프로덕션 변형을 호스트하기 위해 시작할 ML 인스턴스를 지정
    - 프로덕션 변형의 중요 인자
        - Model Name: 위의 "(1)" 에서 생성한 세이지 메이커 모델 경로
        - InstanceType: 배포할 EC2 인스턴스 타입 (에: ml.m4.2xlarge)
        - InitialInstanceCount: EC2 인스턴스 갯수
        - InitialVariantWeight: 프덕뎍션 변형에 보낼 트래픽 비율
    - [알림] 프러덕션 변형은 1개 이상을 정의할 수 있으면, 각각의 프러뎍션 변형에서 InitialVariantWeight 를 통한 트래픽을 조절 합니다.


- 상세 내용은 여기를 참조 하세요. --> [CreateEndpointConfig](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/APIReference/API_CreateEndpointConfig.html)            
            
아래와 같은 코드를 가지고 endpoint configuration을 생성 합니다.
```python
    create_ep_config_response = sagemaker_boto_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            'InstanceType': instance_type,
            'InitialVariantWeight': 1,
            'InitialInstanceCount': 1,
            'ModelName': model_name,
            'VariantName': 'AllTraffic'
        }]
    )
```
아래는 endpoint configuration 생성한 예시 입니다.

![endpoint-config-1.png](img/endpoint-config.png)



## (3) HTTPS 엔드포인트 생성
엔드포인트의 생성은 엔드포인트 이름을 명시하고, 생성한 endpoint_config_name 를 기술하면 됩니다.
```python
    create_endpoint_response = sagemaker_boto_client.create_endpoint(
        EndpointName=args.endpoint_name,
        EndpointConfigName=endpoint_config_name)
```

## [참고] 프로덕션 환경에서 A/B 테스팅 모델
- 두개의 모델을 사용한 A/B 테스팅 예시


![hosting-ABtesting.png](img/hosting-ABtesting.png)

### 3개의 프더덕트 변형 예시
- 아래의 그림 예시는, 프러덕트 변형이 각각 다름 모델 및 ML 인스턴스를 가지고 있습니다. Endpoint의 트래픽을 70%, 20%, 10% 씩을 처리하게 구성되어 있습니다. 이러한 구성은 특정 모델에 대한 A/B Testing 환경 구성에 주로 사용됩니다.
- 상세한 내용은 여기를 참조하세요. --> [프로덕션 환경에서 테스트 모델](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-ab-testing.html)
- 블로그 --> [A/B Testing ML models in production using Amazon SageMaker](https://aws.amazon.com/ko/blogs/machine-learning/a-b-testing-ml-models-in-production-using-amazon-sagemaker/)

![model-traffic-distribution.png](img/model-traffic-distribution.png)



---
# 1. 기본 세이지 메이커 호스트 및 로컬 세션 정의

In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
role = sagemaker.get_execution_role()

%store -r 

#### 세이지 메이커 호스트 세션 정의

In [2]:

import sagemaker

# 세이지 메이커의 호스트 모드에서 사용할 세션
sagemaker_session = sagemaker.session.Session()


#### 세이지 메이커 로컬 세션 정의

In [3]:
# 로컬 노트북 인스턴스에서 사용할 로컬 세션
local_session = sagemaker.local.LocalSession()
local_session.config = {'local' : {'local_code':True}}

# 2. 엔드포인트 배포 스텝 개발 및 실행

# 2.1 개요


### (1) 로컬 노트북 인스턴스에서 다커 컨테이너로 엔드포인트를 생성
- 이 단계의 목적은 로컬 노트북 인스턴스의 다커 컨테이터 안에서 로직 확인 및 디버깅을 위해서 진행 합니다.
    - 이 노트북에서는 예제에 포함이 안되어 있지만, 주로 추론 코드인 inference.py 의 로직 검증을 위해서 사용 합니다.
        - 상세 사항은 여기를 참조 하세요.--> [SageMaker Python SDK: Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html)

- 로컬 노트북 인스턴스에서 엔드포인트를 생성하기 위해서, 조금 더 쉬운 방법인 Python SDK를 통해서 모델을 생성 (sagemaker.model.Model() 사용)하고, Model.deploy() 를 통해서 엔드포인트를 바로 생성합니다.


### (2) 로컬에서 Endpoint 생성 스크립트 실행 (AWS SDK Boto3 API 이용)     
- 이 단계의 목적은 src/deploy_model.py 스크립트의 로직확인을 위함 입니다. 
- src/deploy_model.py 를 로컬에서 실행하여 세이지 메메이커 호스팅 엔드포인트를 생성합니다.
    - 실행이 완료 되면 세이지 메이커의 호스트 엔드포인트가 생성이 됩니다. (로컬 노트북 인스턴스에서 생성되는 것이 아님)

    
### (3) SageMaker Model Building Pipeline 에서 엔드포인트 생성을 수행합니다.
- 최종적으로 src/deploy_model.py 이 SageMaker Model Building Pipeline  에서 실행 됩니다.
- 상세 사항은 여기에서 확인 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](img/https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)
    

---
# 2.2 실행
## (1) 로컬 노트북 인스턴스에서 다커 컨테이너로 엔드포인트를 생성
이 과정은 아래의 7단계로 진행이 됩니다.

- [1] instance_type 을 로컬로 지정
    - 'local' 로 지정하여 사용합니다.
- [2] 세이지 메이커 모델 생성
    - 세이지 메이커 모델을 Python SDK 로 생성합니다.
- [3] 로컬 앤드포인트 생성
    - Python SDK 인 Model.doploy() API 사용. 
        - API 링크 --> [Inference API: Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#)
- [4] 로컬 프리딕터 생성
    - 앤드포인트와 통신하면서 추론 요청의 데이터(paylaod) 대해서 직렬화(Serialization) 후에 엔드포인트에 전달 및 엔드포인트의 결과를 역직렬화(Deserialization) 을 하여 전달 하는 역할을 함.
    - Client Application <-> Predictor <-> Endpoint
        - API 링크 --> [Inference API: Predictors](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html)
- [5] 테스트 세트 데이터 로드
    - 추론을 할 데이터 로딩
- [6] 추론에 사용할 Payload 생성 및 추론
    - 추론 데이터를 text/csv 형태의 payload 로 만들고 프리딕터에 추론 요청을 하여 결과를 받음
- [7] 로컬 엔드 포인트 삭제
    - 이 단계에서 생성한 세이지 메이커 모델, 앤프로인트 컨피그, 앤드포인트 삭제


### [1] instance_type 을 로컬로 지정
- instance_type = 'local' 
    - 로컬 모드를 실행하는 지시자 임.
- local_endpoint_name 의 이름을 지정 합니다.    


In [4]:
from datetime import datetime
suffix = datetime.now().microsecond

In [5]:
instance_type='local'
local_endpoint_name = 'local-endpoint-' + str(suffix)
print("local_endpoint_name: ", local_endpoint_name)


local_endpoint_name:  local-endpoint-605527


### [2] 세이지 메이커 모델 생성

In [6]:
from datetime import datetime
suffix = datetime.now().microsecond
print("suffix: ", suffix)

from sagemaker.model import Model

local_model = Model(
    image_uri=image_uri,
    model_data= train_model_artifact,
    sagemaker_session= local_session,
    role=role
)

suffix:  115651


### [3] 로컬 앤드포인트 생성

에러가 발생시에 하나의 경우로서 이미 로컬 앤드포인트가 실행중이어서 에러가 날 수 있습니다. 아래와 같이 에러 발생하면, 다커 컨테이너를 중지 시켜 주세요.

- 에러 케이스1
    - 아래와 같이 에러가 발생한 경우에 로컬머신에 다커 컨테이너가 이미 실행 상태일 수 있습니다.
```
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 827, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 887, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1
```
- 아래의 그럼 처럼 실행 중인 다커 컨테이너를 확인하고, 중지 시켜주시기 바랍니다.
- ![docke_error_01.png](img/docke_error_01.png)

아래 셀을 실행 하시고, 가장 밑의 로그에 아래와 유사하게 나와야 합니다. (다커 컨테이너에서 웹 서버를 실행하고, HTTP Get() 요청싱에 200의 리턴 코드를 받은 경우 임)

`!gvvsnvcnps-algo-1-kkbax | 172.18.0.1 - - [10/Jul/2021:06:40:10 +0000] "GET /ping HTTP/1.1" 200 0 "-" "python-urllib3/1.26.5"`

In [7]:
_ = local_model.deploy(initial_instance_count = 1,
                                     instance_type = instance_type, 
                                     endpoint_name = local_endpoint_name
            )

Attaching to 7ifkzzcgjh-algo-1-7grkv
7ifkzzcgjh-algo-1-7grkv | [2021-08-03:07:30:40:INFO] No GPUs detected (normal if no gpus installed)
7ifkzzcgjh-algo-1-7grkv | [2021-08-03:07:30:40:INFO] No GPUs detected (normal if no gpus installed)
7ifkzzcgjh-algo-1-7grkv | [2021-08-03:07:30:40:INFO] nginx config: 
7ifkzzcgjh-algo-1-7grkv | worker_processes auto;
7ifkzzcgjh-algo-1-7grkv | daemon off;
7ifkzzcgjh-algo-1-7grkv | pid /tmp/nginx.pid;
7ifkzzcgjh-algo-1-7grkv | error_log  /dev/stderr;
7ifkzzcgjh-algo-1-7grkv | 
7ifkzzcgjh-algo-1-7grkv | worker_rlimit_nofile 4096;
7ifkzzcgjh-algo-1-7grkv | 
7ifkzzcgjh-algo-1-7grkv | events {
7ifkzzcgjh-algo-1-7grkv |   worker_connections 2048;
7ifkzzcgjh-algo-1-7grkv | }
7ifkzzcgjh-algo-1-7grkv | 
7ifkzzcgjh-algo-1-7grkv | http {
7ifkzzcgjh-algo-1-7grkv |   include /etc/nginx/mime.types;
7ifkzzcgjh-algo-1-7grkv |   default_type application/octet-stream;
7ifkzzcgjh-algo-1-7grkv |   access_log /dev/stdout combined;
7ifkzzcgjh-algo-1-7grkv | 
7ifkzzcgjh-algo

### [4] 로컬 프리딕터 생성

- HTTPS 프로토콜로 엔드포인트에 요청 및 반환시에는 Sequence of Bytes의 형태로 데이터가 전송됩니다. Predictor 같은 중간에 오브젝트를 두면, 입력 데이터의 직렬화 및 결과 데이터의 역직렬화를 합니다. 
- 프리딕터의 주요 인자는 `endpoint_name, session, serializer, deserializer 입니다.

- 참고 사항: 
    - [Predictors](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html)
    - [SageMaker Python SDK: Deserializers](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#)

In [8]:
from sagemaker.deserializers import CSVDeserializer
csv_deserializer = CSVDeserializer(accept='text/csv') # 디폴트가 accept가 'text/csv' 이지만 직관적 이유로 기술함.

def get_predictor(endpoint_name, session, csv_deserializer):
    '''
    predictor = get_predictor(endpoint_name, session, csv_deserializer)    
    '''
    predictor = sagemaker.predictor.Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session= session, # 세션 할당: 로컬 세션 혹은 세이지 메이커 호스트 세션
        deserializer = csv_deserializer, # byte stream을 csv 형태로 변환하여 제공        
    )
    return predictor

predictor = get_predictor(local_endpoint_name, local_session, csv_deserializer)


### [5] 테스트 세트 데이터 로드
- 기존에 전처리된 테스트 데이터를 로딩해서 추론에 사용합니다.

In [9]:
import os

test_preproc_dir_artifact_file = os.path.join(test_preproc_dir_artifact, 'test.csv')

7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3253] [INFO] Booting worker with pid: 3253


In [10]:
test_df = pd.read_csv(test_preproc_dir_artifact_file)
test_df.head(2)

7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3379] [INFO] Booting worker with pid: 3379
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3378] [INFO] Booting worker with pid: 3378
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3383] [INFO] Booting worker with pid: 3383
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3556] [INFO] Booting worker with pid: 3556
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3692] [INFO] Booting worker with pid: 3692
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3711] [INFO] Booting worker with pid: 3711
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3726] [INFO] Booting worker with pid: 3726
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:42 +0000] [3779] [INFO] Booting worker with pid: 3779
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:43 +0000] [3911] [INFO] Booting worker with pid: 3911
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:43 +0000] [3914] [INFO] Booting worker with pid: 3914
7ifkzzcgjh-algo-1-7g

fraud  vehicle_claim  total_claim_amount  customer_age  months_as_customer  \
0      0   23901.432507        36201.432507            56                 118   
1      0   29095.295736        73695.295736            36                  71   

   num_claims_past_year  num_insurers_past_5_years  policy_deductable  \
0                     0                          1                750   
1                     0                          1                750   

   policy_annual_premium  customer_zip  ...  collision_type_missing  \
0                   3000         98039  ...                       0   
1                   2650         95307  ...                       0   

   incident_severity_Major  incident_severity_Minor  \
0                        0                        0   
1                        0                        0   

   incident_severity_Totaled  authorities_contacted_Ambulance  \
0                          1                                0   
1                          1                                0   

   authorities_contacted_Fire  authorities_contacted_None  \
0                           0                           0   
1                           0                           0   

   authorities_contacted_Police  police_report_available_No  \
0                             1                           0   
1                             1                           0   

   police_report_available_Yes  
0                            1  
1                            1  

[2 rows x 59 columns]

7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:43 +0000] [5012] [INFO] Booting worker with pid: 5012


### [6] 추론에 사용할 Payload 생성 및 추론
- 위의 테스트 데이터를 통해서 엔드포인트에 전달할 CSV 형태의 String을 생성합니다. (payload).
- payload 를 엔드포인트에 제공하면, 확률값을 0.0726071447134018 을 리턴합니다. 
- 보통 확률값이 0.5 보다 작으면 0 (Non-Fruad), 0.5 보다 크면 1 (Fruad) 로 변환해서 예측값으로 사용합니다.

```
payload: 
 6038.102399076349,15838.10239907635,39,64,0,1,750,2750,95660,2012,1,0,0,9800,1,9,2,16,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0
Reponse: 
[['0.0726071447134018']]

```

In [11]:
from IPython.display import display as dp
 
def get_payload(sample, label_col = 'fraud', verbose=False):    
    '''
    아래왁 같이 ',' 형태의 문자형을 리턴함.
    0,0,750,2,0,2,0,2,1,16596.0,1,18,0,59,1,0,0,4500.0,0,0,1,1,0,52,2020,3,0,0,0,2,1,0,0,0,0,0,0,10,12096.0,1,3000,1,0,1,0    
    '''

    sample = sample.drop(columns=[label_col]) # 레이블 제거

    payload = sample.to_csv(header=None, index=None).splitlines() # 
    payload = payload[0]

    if verbose:
        #dp(sample)
        # print("payload length: \n", len(payload))    
        print("pay load type: ", type(payload))
        print("payload: \n", payload)
    
    return payload

def predict(predictor, payload):
    '''
    프리딕터에 콤마 분리된 문자형과 ContentType을 'text/csv' 로 제공
    참고:
        CSVDeserializer 를 사용하지 않으면 byte stream 으로 제공되기에, 아래와 같이 디코딩 하여 사용함.
        result = float(result.decode())
    '''

    pay_load = get_payload(test_df, label_col='fraud', verbose=True)
    result = predictor.predict(pay_load, initial_args = {"ContentType": "text/csv"})
    
    return result



In [12]:
sample = test_df.sample(1)
payload = get_payload(test_df, label_col = 'fraud', verbose=False)
pred_prob = predict(predictor, payload)
pred_prob

7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:43 +0000] [5014] [INFO] Booting worker with pid: 5014
pay load type:  <class 'str'>
payload: 
 23901.43250714285,36201.43250714285,56,118,0,1,750,3000,98039,2012,3,3,1,12300,9,12,3,14,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1
7ifkzzcgjh-algo-1-7grkv | [2021-08-03:07:30:43:INFO] No GPUs detected (normal if no gpus installed)
7ifkzzcgjh-algo-1-7grkv | [2021-08-03 07:30:43 +0000] [5082] [INFO] Booting worker with pid: 5082
7ifkzzcgjh-algo-1-7grkv | [2021-08-03:07:30:43:INFO] Determined delimiter of CSV input is ','


[['0.10092484205961227']]

7ifkzzcgjh-algo-1-7grkv | 172.21.0.1 - - [03/Aug/2021:07:30:43 +0000] "POST /invocations HTTP/1.1" 200 19 "-" "python-urllib3/1.26.5"


### [7] 로컬 엔드 포인트 삭제
- 위에서 생성한 세이지 메이커 모델, enpoint config, endpoint를 삭제 합니다.

In [13]:
def delete_endpoint(client, endpoint_name ,is_del_model=True):
    '''
    model, EndpointConfig, Endpoint 삭제
    '''
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']    

#     print("EndpointConfigName: \n", EndpointConfigName)
#     print("model_name: \n", model_name)    

    if is_del_model: # 모델도 삭제 여부 임.
        client.delete_model(ModelName=model_name)    
        
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')    


In [14]:
client = sagemaker.local.LocalSagemakerClient() # 로컬 세션 생성
delete_endpoint(client, local_endpoint_name )

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: sagemaker-xgboost-2021-08-03-07-30-37-039
--- Deleted endpoint: local-endpoint-605527
--- Deleted endpoint_config: local-endpoint-605527


---

## (2) 로컬에서 Endpoint 생성 스크립트 실행 (Boto3 API 이용)

### [중요] 아래는 약 8분 정도의 시간이 소요 됩니다.
### 다음과 같은 단계를 실행 합니다.
- [1] 배포 스크립트 확인
- [2] 배포 스크립트 커맨드 인자 정의 및 확인
- [3] 배포 스크립트 실행
- [4] 추론 테스트 (프리딕터 생성, payload 생성, 추론 실행)
- [5] 앤드포인트 컨피그, 앤드포인트 삭제


### [1] 배포 스크립트 확인

In [15]:
!pygmentize src/deploy_model.py # 코드를 보기 위해서 주석을 제거하세요.

import time
import boto3
import argparse
import sys, os

import logging
import logging.handlers

def _get_logger():
    '''
    로깅을 위해 파이썬 로거를 사용
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  

logger = _get_logger()


#########################################
## 커맨드 인자 처리
#########################################

# Parse argument variables passed via the DeployModel processing step
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str)
parser.add_argument('--region', type=str, default="ap-northeast-2")
parser.add_argument('--endpoint_instance_type', type=str, default='ml.t3.medium')
parser.add_argument('--endpoint_name', type=str)
args = parser.pars

### [2] 배포 스크립트 커맨드 인자 정의 및 확인

In [16]:
import sagemaker
region = sagemaker.Session().boto_region_name
endpoint_name = 'fraud2scratch-' + str(suffix)
endpoint_instance_type = 'ml.t2.xlarge'

In [17]:
print("Using AWS Region: {}".format(region))
print("sagemaker_model: ", sagemaker_model)
print("endpoint_instance_type: ", endpoint_instance_type)
print("endpoint_name: ", endpoint_name)

Using AWS Region: ap-northeast-2
sagemaker_model:  pipelines-ptvhrz2rc2jp-fraudscratchmodel-oeukwysix7
endpoint_instance_type:  ml.t2.xlarge
endpoint_name:  fraud2scratch-115651


### [3] 배포 스크립트 실행
- [중요] 약 8분 소요 됩니다. 기다려 주세요.

In [18]:
%%sh -s "$sagemaker_model" "$region" "$endpoint_instance_type" "$endpoint_name" 
python src/deploy_model.py \
--model_name $1 \
--region $2 \
--endpoint_instance_type $3 \
--endpoint_name $4



#############################################
args.model_name: pipelines-ptvhrz2rc2jp-fraudscratchmodel-oeukwysix7
args.region: ap-northeast-2
args.endpoint_instance_type: ml.t2.xlarge
args.endpoint_name: fraud2scratch-115651
Creating endpoint
Endpoint status is creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService


위의 `[3] 배포 스크립트 실행` 수행 후에 아래 세이지 메이커 컨솔에 가셔서 앤드로인트가 생성 중인지를 확인 해보세요.
![console_create_endpoint-1.png](img/console_create_endpoint.png)

### [4] 추론 테스트 (프리딕터 생성, payload 생성, 추론 실행)

In [19]:
predictor = get_predictor(endpoint_name, sagemaker_session, csv_deserializer)

In [20]:
import numpy as np
sample = test_df.sample(1)
payload = get_payload(test_df, label_col = 'fraud', verbose=False)
pred_prob = predict(predictor, payload)
pred_prob

pay load type:  <class 'str'>
payload: 
 23901.43250714285,36201.43250714285,56,118,0,1,750,3000,98039,2012,3,3,1,12300,9,12,3,14,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1


[['0.10092484205961227']]

### [5] 앤드포인트 컨피그, 앤드포인트 삭제
- is_del_model=False  를 통해서 생성한 모델은 삭제 하지 않습니다. 아래 파이프라인을 통한 엔드포인트 생성시에 모델을 사용하기 위함입니다.

In [21]:
import boto3
boto3_client = boto3.client('sagemaker')
delete_endpoint(boto3_client, endpoint_name, is_del_model=False )
# delete_endpoint(client, local_endpoint_name )

--- Deleted model: pipelines-ptvhrz2rc2jp-fraudscratchmodel-oeukwysix7
--- Deleted endpoint: fraud2scratch-115651
--- Deleted endpoint_config: pipelines-ptvhrz2rc2jp-fraudscratchmodel-oeukwysix7-config


---
## (3) SageMaker Model Building Pipeline 에서 엔드포인트 생성을 수행합니다.
### [중요] 이 단계는 약 20분 정도 소요 됩니다.

이 단계는 내부적으로 크게 아래 두가지 요소로 진행 됩니다.
- deploy_step 단계로 이름 지어진 Processing Step 의 진행  (EC2 생성), 
- 이후에 EC2 안에서 src/deploy_model.py 실행 되면서 엔드포인트 생성에 시간 (약 8분) 이 걸립니다.


### 모델 빌딩 파이프라인 변수 생성

In [22]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

## 배포 스텝 정의
[중요] `pipeline_endpoint_name` 에 '_' 언데바를 넣으면 에러가 납니다. '-' 대시는 가능합니다.

In [23]:

local_deploy_code_path = 'src/deploy_model.py'
s3_deploy_code_path = f"s3://{default_bucket}/{project_prefix}/code"
s3_deploy_code_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_deploy_code_path, 
    desired_s3_uri=s3_deploy_code_path,
)
print("s3_deploy_code_uri: ", s3_deploy_code_uri)

pipeline_endpoint_name = 'pipeline-endpoint-'  + str(suffix)

s3_deploy_code_uri:  s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/code/deploy_model.py


In [24]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role= role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='fraud-scratch-deploy-model',
    sagemaker_session=sagemaker_session)


deploy_step = ProcessingStep(
    name='DeployModel',
    processor=deploy_model_processor,
    job_arguments=[
        "--model_name", sagemaker_model, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--endpoint_name", pipeline_endpoint_name
    ],
    code=s3_deploy_code_uri)

### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [25]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
    ],
    steps=[deploy_step],
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [26]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'DeployModel',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.t3.medium',
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ContainerArguments': ['--model_name',
      'pipelines-ptvhrz2rc2jp-fraudscratchmodel-oeukwysix7',
      '--region',
      'ap-northeast-2',
      '--endpoint_instance_type',
      'ml.t2.xlarge',
      '--endpoint_name',
      'pipeline-endpoint-115651'],
     'ContainerEntrypoint': ['py

#### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 SageMaker Pipelines 서비스에 제출하여 파이프라인을 생성하거나 파이프라인이 이미 존재하면 파이프라인 정의를 업데이트합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [27]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'ResponseMetadata': {'RequestId': '68961a13-fd91-434b-909e-c6d4f242a49a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '68961a13-fd91-434b-909e-c6d4f242a49a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Tue, 03 Aug 2021 07:37:18 GMT'},
  'RetryAttempts': 0}}

In [28]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [29]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step/execution/clq5s5jkqi8v',
 'PipelineExecutionDisplayName': 'execution-1627976239028',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 8, 3, 7, 37, 18, 964000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 3, 7, 37, 18, 964000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'c66f7a0b-4a92-421c-b577-02126fe5a09f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c66f7a0b-4a92-421c-b577-02126fe5a09f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '437',
   'date': 'Tue, 03 Aug 2021 07:37:18 GMT'},
  'RetryAttempts': 0}}

In [30]:
execution.wait()

In [31]:
execution.list_steps()

[{'StepName': 'DeployModel',
  'StartTime': datetime.datetime(2021, 8, 3, 7, 37, 19, 532000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 3, 7, 55, 10, 930000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:processing-job/pipelines-clq5s5jkqi8v-deploymodel-w0f1rkhaq9'}}}]

### [옵션] SageMaker Studio에서 확인하기
- 이전의 3.1.Preprocesing-Pipleline 노트북에서 언급 되었듯이, SageMaker Studio 에서도 확인이 가능합니다. 이전 노트북을 참조 해주세요.


### 배포 파이프라인 추론 테스트

In [32]:
import boto3
sagemaker_boto_client = boto3.client('sagemaker')


def is_available_endpoint(endpoint_name, verbose=False):
    '''
    Return True if endpoint is in service, otherise do False
    '''
    response = sagemaker_boto_client.list_endpoints(NameContains=endpoint_name)
    #existing_endpoints = sagemaker_boto_client.list_endpoints(NameContains=endpoint_name)['Endpoints']
    
    if verbose:
        print("Response: \n", response)

            
    EndpointStatus = response['Endpoints'][0]['EndpointStatus']
    if verbose:
        print("EndpointStatus: ", EndpointStatus)
    
        
    if EndpointStatus == 'InService':
        return True
    else:
        return False



if is_available_endpoint(pipeline_endpoint_name, verbose=False):
    pipeline_predictor = get_predictor(pipeline_endpoint_name, sagemaker_session, csv_deserializer)
    sample = test_df.sample(1)
    payload = get_payload(test_df, label_col = 'fraud', verbose=False)
    pred_prob = predict(pipeline_predictor, payload)
    print("pred_prob: \n", pred_prob)
else:
    print("Wait for creating an endpoint")


pay load type:  <class 'str'>
payload: 
 23901.43250714285,36201.43250714285,56,118,0,1,750,3000,98039,2012,3,3,1,12300,9,12,3,14,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1
pred_prob: 
 [['0.10092484205961227']]


### 엔드포인트 삭제

In [33]:
delete_endpoint(boto3_client, pipeline_endpoint_name, is_del_model = True )

--- Deleted model: pipelines-ptvhrz2rc2jp-fraudscratchmodel-oeukwysix7
--- Deleted endpoint: pipeline-endpoint-115651
--- Deleted endpoint_config: pipelines-ptvhrz2rc2jp-fraudscratchmodel-oeukwysix7-config
